In [46]:
import os

import numpy as np
import matplotlib.pyplot as plt


import tensorflow as tf
import tensorflow.keras

from sklearn.metrics import roc_auc_score
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

# Load Data

In [130]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

#x_train = x_train.reshape(60000, 784).astype("float32") / 255
#x_test = x_test.reshape(10000, 784).astype("float32") / 255

x_train = x_train.reshape(60000, 784) / 255.0
x_test = x_test.reshape(10000, 784) / 255.0

y_train = y_train.reshape(60000,1)
y_test = y_test.reshape(10000,1)
#x_train, x_test = x_train / 255.0, x_test / 255.0

In [75]:
y_train.shape

(60000, 1)

In [68]:
'''tensorflow dataset
x_train = tf.data.Dataset.from_tensor_slices(x_train)
x_train = x_train.shuffle(buffer_size=1024).batch(64)


y_train = tf.data.Dataset.from_tensor_slices(y_train)
y_train = y_train.shuffle(buffer_size=1024).batch(64)




'''

(60000, 784)

# Define Model

In [3]:
# class Deep_SVDD(Model):
#     def __init__(self, hidden1, hidden2, latent, input_dim):
#         super(Deep_SVDD, self).__init__()
# #         self.encoder_layer1 = layers.Dense(hidden1, activation = 'relu')
# #         self.encoder_layer2 = layers.Dense(hidden2, activation = 'relu')

# #         self.latent = layers.Dense(latent, activation = 'relu')

# #         self.decoder_layer1 = layers.Dense(hidden2, activation = 'relu')
# #         self.decoder_layer2 = layers.Dense(hidden1, activation = 'relu')
        
#         self.encoder = tf.keras.Sequential([
#             layers.Dense(hidden1, activation='relu'),
#             layers.Dense(hidden2, activation='relu')
#         ])
        
#         self.latent = layers.Dense(latent, activation = 'relu')
        
#         self.decoder = tf.keras.Sequential([
#             layers.Dense(hidden2, activation='relu'),
#             layers.Dense(hidden1, activation='linear'),
#         ])
        
#         self.input_dim = input_dim
#         self.input_layer = layers.Input(shape = input_dim)
#         self.output_layer = self.call(self.input_layer)
#         self.c = 0
    
#     def build_graph(self):
#         inputs_ = layers.Input(shape = self.input_dim)
#         return Model(inputs=inputs_, outputs=self.call(inputs_))
#         #self._init_graph_network(inputs=self.input_layer,outputs=self.out)
    
#     def center_point(self):
#         #self.c += self.latent
#         pass
#     #def build(self):
#     #    self._is_graph_network = True
#     #      self._init_graph_network(inputs=self.input_layer,outputs=self.output_layer)

#     def call(self, input_data):
#         x = self.encoder(input_data)
#         x = self.latent(x)
#         x = self.decoder(x)
        
#         #return Model(input_data, x)
#         return x

In [110]:
class Pretrain_AutoEncoder(Model):
    def __init__(self, hidden1, hidden2, latent, input_dim):
        super(Pretrain_AutoEncoder, self).__init__()
        self.input_dim = input_dim
        
        self.encoder_layer1 = layers.Dense(hidden1, activation = 'relu', name='encoder1')
        self.encoder_layer2 = layers.Dense(hidden2, activation = 'relu', name='encoder2')

        self.latent = layers.Dense(latent, activation = 'relu', name='latent')

        self.decoder_layer1 = layers.Dense(hidden2, activation = 'relu', name='decoder1')
        self.decoder_layer2 = layers.Dense(hidden1, activation = 'relu', name='decoder2')
        self.outputs = layers.Dense(self.input_dim, activation = 'relu', name = 'outputs')
#         self.encoder = tf.keras.Sequential([
#             layers.Dense(hidden1, activation='relu'),
#             layers.Dense(hidden2, activation='relu')
#         ])
        
        #self.latent = layers.Dense(latent, activation = 'relu')
        
#         self.decoder = tf.keras.Sequential([
#             layers.Dense(hidden2, activation='relu'),
#             layers.Dense(hidden1, activation='linear'),
#         ])
        
    
    def build_graph(self):
        inputs_ = layers.Input(shape=self.input_dim, name = 'inputs')
        return Model(inputs=inputs_, outputs=self.call(inputs_))
        #self._init_graph_network(inputs=self.input_layer,outputs=self.out)
    
    def call(self, input_data):
#         x = self.encoder(input_data)
#         x = self.latent(x)
#         x = self.decoder(x)
        x = self.encoder_layer1(input_data)
        x = self.encoder_layer2(x)
        x = self.latent(x)
        x = self.decoder_layer1(x)
        x = self.decoder_layer2(x)
        x = self.outputs(x)
        
        return x

In [211]:
class DeepSVDD(Model):
    def __init__(self, hidden1, hidden2, latent, input_dim):
        super(DeepSVDD, self).__init__()
        self.encoder_layer1 = layers.Dense(hidden1, activation = 'relu', name='encoder1')
        self.encoder_layer2 = layers.Dense(hidden2, activation = 'relu', name='encoder2')

        self.latent = layers.Dense(latent, activation = 'relu', name='latent')
        
        self.input_dim = input_dim
#         self.encoder = tf.keras.Sequential([
#             layers.Dense(hidden1, activation='relu'),
#             layers.Dense(hidden2, activation='relu')
#         ])
        
#         self.latent = layers.Dense(latent, activation = 'relu')
        
#         self.input_dim = input_dim
#         self.input_layer = layers.Input(shape = input_dim)
#         self.output_layer = self.call(self.input_layer)
    
    def build_graph(self):
        inputs_ = layers.Input(shape = self.input_dim, name = 'inputs')
        return Model(inputs=inputs_, outputs=self.call(inputs_))
        #self._init_graph_network(inputs=self.input_layer,outputs=self.out)
    
    def call(self, input_data):
        x = self.encoder_layer1(input_data)
        x = self.encoder_layer2(x)
        x = self.latent(x)
        #return Model(input_data, x)
        return x

In [26]:
pretrain_weight_path = ['.', 'pretrain_weight']
pretrain_weight_path = '.pretrain_weight'
os.path.join(pretrain_weight_path) + '\\'

'.pretrain_weight\\'

In [41]:
os.path.join(os.path.abspath('.'), 'pretrain_weight_path', '{epoch:03d}_{val_loss:.4f}.hdf5')

'C:\\Users\\HYKP\\PycharmProjects\\bistel\\Deep_SVDD\\pretrain_weight_path\\{epoch:03d}_{val_loss:.4f}.hdf5'

In [42]:
os.path.join(os.path.abspath('.'), 'pretrain_weight_path')

'C:\\Users\\HYKP\\PycharmProjects\\bistel\\Deep_SVDD\\pretrain_weight_path'

In [39]:
a = 0.123213213213213213
f"a: {a:.2f}"

'a: 0.12'

In [195]:
MODEL_SAVE_DIR_PATH = os.path.join(os.path.abspath('.'), 'pretrain_weight') # '.\\pretrain_weight\\'

In [230]:
a = ae
if a:
    print(123)

123


In [341]:
MODEL_SAVE_DIR_PATH = os.path.join(os.path.abspath('.'), 'pretrain_weight') # '.\\pretrain_weight\\'

class Trainer:
    def __init__(self, args, pretrain_model, svdd_model, train, test):
        self.args = args
        
        if pretrain_model:
            self.pretrain_model = pretrain_model
        self.svdd_model = svdd_model
        
        self.x_train = train[0]
        self.y_train = train[1]
        
        self.x_test = test[0]
        self.y_test = test[1]
    
    def start_check(self):
        if not os.path.exists('pretrain_weight'):
            os.mkdir('pretrain_weight')
        
    
    def pretrain_ae(self):
        #ae = Pretrain_AutoEncoder(350, 50, 2, 784).build_graph()
        ae = self.pretrain_model.build_graph()
        loss_object=tf.keras.losses.MeanSquaredError() #SVDD
        #loss_object = tf.keras.losses.SparseCategoricalCrossentropy() #mnist
        optimizer = tf.keras.optimizers.Adam()
        
        #ae.compile(optimizer=optimizer, loss=loss_object, metrics=['mae']) #SVDD
        ae.compile(optimizer=optimizer, loss=loss_object, metrics=['mae']) #mnist
        hist = ae.fit(x=self.x_train, y=self.x_train, batch_size=self.args['batch'], validation_split=0.2, epochs=self.args['epochs'])
        
        encoder_part = Model(ae.get_layer('inputs').input, ae.get_layer('latent').output)
        
        encoder_part.save_weights(os.path.join(MODEL_SAVE_DIR_PATH, 'pretrain_ae.hdf5'))
        
        return hist
    
    def set_c(self, eps=0.1):
        """Initializing the center for the hypersphere"""
        model = self.svdd_model.build_graph()
        if self.args['pretrain'] == True:
            model.load_weights(os.path.join(MODEL_SAVE_DIR_PATH, 'pretrain_ae.hdf5'))
        
        
        #x_train = tf.data.Dataset.from_tensor_slices(self.x_train)
        #x_train = x_train.shuffle(buffer_size=1024).batch(64)        
        
        z_list = []
        z_list_temp = []
        #new = np.array_split(x_train, 938, axis = 0)
        #for k, x in enumerate(x_train):
        #    z_list.append(model.predict(x))
        
        z = model.predict(self.x_train)        
        #z_list = np.concatenate(z_list)
        #c = z_list.mean(axis = 0)
        c = z.mean()
        
        c[(abs(c) < eps) & (c < 0)] = -eps
        c[(abs(c) < eps) & (c > 0)] = eps
        
        return c
        
        '''
        for key, x in enumerate(self.x_train):
            if key % self.args['batch'] == 0:
                z_list.append(z_list_temp)
                z_list_temp = []
            else:
                z = model.predict(np.expand_dims(x, axis = 0))
                z_list.append(z)
        print(len(z_list))
        print(len(z_list[0]))
        print(len(z_list[-1]))
        '''
        '''
        with torch.no_grad():
            for x, _ in dataloader:
                x = x.float().to(self.device)
                z = model.encoder(x)
                z_.append(z.detach())
        z_ = torch.cat(z_)
        c = torch.mean(z_, dim=0)
        c[(abs(c) < eps) & (c < 0)] = -eps
        c[(abs(c) < eps) & (c > 0)] = eps
        return c
        '''
    
    
    def deep_svdd_train(self):
        pass
        
            
        
  
    def eval_step(self):
        pass
        #prediction = 

In [251]:
model = svdd.build_graph()
model.load_weights(os.path.join(MODEL_SAVE_DIR_PATH, 'pretrain_ae.hdf5'))


In [296]:
import math
l = len(x_train)
math.ceil(60000/64)

938

In [305]:
for i in np.array_split(x_train, 938, axis = 0):
    print(i.shape)
    print(type(i))
    break

(64, 784)
<class 'numpy.ndarray'>


In [362]:
c = model.predict(x_train).mean()

In [363]:
c

4.4077797

In [364]:
eps = 0.1

In [365]:
c[(abs(c) < eps) & (c < 0)]

array([], dtype=float32)

In [355]:
c = np.array(c)

In [366]:
c

4.4077797

In [358]:
c[(abs(c) < eps) & (c < 0)] = -eps

In [367]:
c = np.array(c)
c[(abs(c) < eps) & (c < 0)] = -eps
c[(abs(c) < eps) & (c > 0)] = eps

In [368]:
c

array(4.4077797, dtype=float32)

In [322]:
z_list = []
for k, x in enumerate(np.array_split(x_train, 938, axis = 0)):
    z_list.append(model.predict(x))

In [339]:
z_list = np.concatenate(z_list)
c = z_list.mean(axis = 0)

ValueError: zero-dimensional arrays cannot be concatenated

In [256]:
model.summary()

Model: "model_51"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
encoder1 (Dense)             (None, 500)               392500    
_________________________________________________________________
encoder2 (Dense)             (None, 350)               175350    
_________________________________________________________________
latent (Dense)               (None, 2)                 702       
Total params: 568,552
Trainable params: 568,552
Non-trainable params: 0
_________________________________________________________________


In [273]:
np.expand_dims(x_train[0], axis = 0).shape
x_train[0][np.newaxis,:].shape

(1, 784)

In [264]:
model.predict(x_train[:1])

array([[4.387509, 4.905375]], dtype=float32)

In [274]:
for k, value in enumerate(x_train):
    model.predict(np.expand_dims(value, axis = 0))
    break

In [186]:
#svdd = Pretrain_AutoEncoder(500, 350, 2, 784)
#svdd.build((None,784))

In [187]:
#svdd.summary()

In [188]:
#os.environ["PATH"] += os.pathsep + "C:/Program Files/Graphviz/bin/"
#tf.keras.utils.plot_model(svdd.build_graph(), show_shapes=True, show_layer_names=True, rankdir='TB', dpi=100, expand_nested=True)

In [189]:
#svdd.build_graph().summary()

In [342]:
args = {'epochs':2,
       'batch': 64,
       'pretrain':True,
       'tensorflow_data_type':False}

ae = Pretrain_AutoEncoder(500, 350, 2, 784)
svdd = DeepSVDD(500, 350, 2, 784)

trainer = Trainer(args, pretrain_model=ae, svdd_model=svdd, train=[x_train, x_train], test = [x_test, x_test])
trainer.start_check()

In [319]:
#hist = trainer.pretrain_ae()

In [343]:
trainer.set_c()

TypeError: 'numpy.float32' object does not support item assignment